<a href="https://colab.research.google.com/github/ahmedyahia01/ComfyUI-Colab/blob/main/ComfyUI_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title الخلية 1: الاتصال بـ Drive والتحقق من الإعدادات
import os
import sys
from google.colab import drive
from google.colab import userdata

print("--- 1. ربط Google Drive ---")
# سيطلب منك هذا الأمر السماح بالوصول إلى Google Drive الخاص بك
drive.mount('/content/drive')

print("\n--- 2. التحقق من مفتاح الوصول (Access Token) ---")
# سيقرأ هذا الكود مفتاح الوصول الذي حفظته في قسم الأسرار (Secrets)
try:
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token or not hf_token.startswith("hf_"):
        print("❌ خطأ: مفتاح الوصول 'HF_TOKEN' غير موجود أو غير صالح.")
        print("الرجاء التأكد من إدخاله بشكل صحيح في قسم 🔑 الأسرار على اليسار.")
        sys.exit("إيقاف التنفيذ: مفتاح الوصول مطلوب.")
    else:
        print("✅ تم العثور على مفتاح الوصول الصالح.")
except Exception:
    print("❌ فشل في قراءة مفتاح الوصول. تأكد من أنك قمت بتفعيله لدفتر العمل هذا.")
    sys.exit("إيقاف التنفيذ: فشل قراءة المفتاح.")


# --- 3. إعداد المسارات الرئيسية ---
print("\n--- 3. إعداد المسارات ومجلد العمل ---")
BASE_DIR = "/content/drive/MyDrive/ComfyUI_Course_Project"
COMFYUI_DIR = os.path.join(BASE_DIR, "ComfyUI")
os.makedirs(BASE_DIR, exist_ok=True)
print(f"✅ سيتم إعداد كل شيء في المجلد: {BASE_DIR}")

print("\n🎉 اكتملت خلية الإعداد. أنت الآن جاهز لتشغيل خلية التثبيت والتحميل.")

--- 1. ربط Google Drive ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- 2. التحقق من مفتاح الوصول (Access Token) ---
✅ تم العثور على مفتاح الوصول الصالح.

--- 3. إعداد المسارات ومجلد العمل ---
✅ سيتم إعداد كل شيء في المجلد: /content/drive/MyDrive/ComfyUI_Course_Project

🎉 اكتملت خلية الإعداد. أنت الآن جاهز لتشغيل خلية التثبيت والتحميل.


In [ ]:

# @title الخلية 2: التثبيت والتحميل الشامل (نسخة نهائية ومستقرة)

import os
import sys
import shutil
from google.colab import userdata
from huggingface_hub import hf_hub_download

# --- 1. إعداد المسارات ---
print("--- 1/6: إعداد المسارات ---")
BASE_DIR = "/content/drive/MyDrive/ComfyUI_Course_Project"
COMFYUI_DIR = os.path.join(BASE_DIR, "ComfyUI")
checkpoints_dir = os.path.join(COMFYUI_DIR, "models", "checkpoints")
TEMP_DOWNLOAD_DIR = "/content/temp_downloads" # مساحة العمل المؤقتة

os.makedirs(checkpoints_dir, exist_ok=True)
print("✅ المسارات جاهزة.")

# --- 2. التحقق من وجود ComfyUI ---
print("\n--- 2/6: التحقق من وجود ComfyUI ---")
if not os.path.exists(os.path.join(COMFYUI_DIR, "main.py")):
    print(f"📂 جاري تحميل ComfyUI...")
    get_ipython().system(f'git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"')
else:
    print("✅ مجلد ComfyUI موجود بالفعل.")

# --- 3. تثبيت الإضافات (Custom Nodes) ---
print("\n--- 3/6: تثبيت الإضافات (Custom Nodes) ---")
# (الكود هنا كما هو، فهو سليم)
nodes_dir = os.path.join(COMFYUI_DIR, "custom_nodes")
%cd $nodes_dir
custom_nodes_repos = {
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-Impact-Pack": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git",
}
for name, url in custom_nodes_repos.items():
    if not os.path.exists(name):
        print(f"  -> Installing {name}...")
        get_ipython().system(f'git clone {url}')
    else:
        print(f"  -> {name} already exists.")

# --- 4. تحميل النماذج الأساسية (بطريقة مستقرة) ---
print("\n--- 4/6: تحميل النماذج الأساسية (هذه الخطوة هي الأطول) ---")
try:
    hf_token = userdata.get('HF_TOKEN')
    if not hf_token or not hf_token.startswith("hf_"):
        sys.exit("❌ خطأ: مفتاح الوصول 'HF_TOKEN' غير موجود.")
    else:
        print("✅ تم العثور على مفتاح الوصول.")
except Exception:
    sys.exit("❌ فشل قراءة مفتاح الوصول.")

model_filename = "sd3_medium_incl_clips.safetensors"
final_model_path = os.path.join(checkpoints_dir, model_filename)

if os.path.exists(final_model_path) and os.path.getsize(final_model_path) > 5_000_000_000: # تحقق من الحجم
    print(f"✅ نموذج {model_filename} موجود ومكتمل في Google Drive. سيتم التخطي.")
else:
    print(f"📂 نموذج {model_filename} غير موجود أو غير مكتمل، جاري التحضير للتحميل...")

    # الخطوة 0: تنظيف أي تحميلات قديمة تالفة لضمان البدء من الصفر
    print("   -> الخطوة 0/3: 🧹 تنظيف مساحة العمل المؤقتة...")
    if os.path.exists(TEMP_DOWNLOAD_DIR):
        shutil.rmtree(TEMP_DOWNLOAD_DIR)
    os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)
    print("   -> 👍 مساحة العمل نظيفة.")

    try:
        # الخطوة 1: التحميل على المساحة المحلية المؤقتة والسريعة لـ Colab
        print("   -> الخطوة 1/3: 📥 جاري التحميل على مساحة Colab المحلية (قد يستغرق وقتاً طويلاً)...")
        hf_hub_download(
            repo_id="stabilityai/stable-diffusion-3-medium",
            filename=model_filename,
            local_dir=TEMP_DOWNLOAD_DIR,
            local_dir_use_symlinks=False,
            token=hf_token
        )
        print("   -> 👍 اكتمل التحميل المحلي بنجاح.")

        # الخطوة 2: نقل الملف المكتمل إلى Google Drive
        print("   -> الخطوة 2/3: 🚚 جاري نقل الملف إلى Google Drive (قد تستغرق هذه الخطوة بضع دقائق)...")
        source_path = os.path.join(TEMP_DOWNLOAD_DIR, model_filename)
        shutil.move(source_path, final_model_path)
        print("   -> ✅ تم نقل النموذج بنجاح إلى Google Drive!")

        # الخطوة 3: تنظيف المجلد المؤقت بعد النجاح
        print("   -> الخطوة 3/3: 🧹 تنظيف المجلد المؤقت بعد انتهاء العملية...")
        shutil.rmtree(TEMP_DOWNLOAD_DIR)
        print("   -> ✅ اكتملت العملية بنظافة.")

    except Exception as e:
        print(f"   -> ❌ فشل تحميل نموذج SD3. الخطأ: {e}")
        print("   -> تأكد من موافقتك على شروط النموذج في موقعه على Hugging Face.")

# --- 5 & 6: تثبيت متطلبات بايثون ---
print("\n--- 5/6 & 6/6: تثبيت متطلبات بايثون ---")
%cd $COMFYUI_DIR
get_ipython().system('pip install -r requirements.txt --quiet')
req_files = [os.path.join(root, name) for root, _, files in os.walk(nodes_dir) if 'requirements.txt' in files for name in files if name == 'requirements.txt']
for file in req_files:
    print(f"  -> Installing requirements from {os.path.dirname(file).split('/')[-1]}...")
    get_ipython().system(f'pip install -r "{file}" --quiet')

print("\n🎉🎉🎉 اكتملت عملية الإعداد والتحميل الطويلة! 🎉🎉🎉")
print("الآن يمكنك تشغيل الخلية الأخيرة لبدء ComfyUI.")

--- 1/6: إعداد المسارات ---
✅ المسارات جاهزة.

--- 2/6: التحقق من وجود ComfyUI ---
✅ مجلد ComfyUI موجود بالفعل.

--- 3/6: تثبيت الإضافات (Custom Nodes) ---
/content/drive/MyDrive/ComfyUI_Course_Project/ComfyUI/custom_nodes
  -> ComfyUI-Manager already exists.
  -> ComfyUI-Impact-Pack already exists.

--- 4/6: تحميل النماذج الأساسية (هذه الخطوة هي الأطول) ---
✅ تم العثور على مفتاح الوصول.
✅ نموذج sd3_medium_incl_clips.safetensors موجود ومكتمل في Google Drive. سيتم التخطي.

--- 5/6 & 6/6: تثبيت متطلبات بايثون ---
/content/drive/MyDrive/ComfyUI_Course_Project/ComfyUI
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

# @title الخلية 3: التشغيل النهائي المتكامل (تنظيف + تشغيل + انتظار + رابط)

import os
import time
import subprocess
from urllib import request, error

# --- إعدادات أساسية ---
PORT = 8188
MAX_WAIT_SECONDS = 180

# --- دالة لمراقبة وتشغيل السيرفر ---
def wait_for_service(port, timeout=MAX_WAIT_SECONDS):
    """تنتظر حتى يصبح السيرفر متاحًا على البورت المحدد."""
    print(f"⏳ جاري انتظار الخادم ليكون جاهزًا على البورت {port}...", end="")
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            with request.urlopen(f"http://localhost:{port}/", timeout=1):
                print("\n✅ الخادم يعمل الآن وجاهز للاستقبال.")
                return True
        except (error.URLError, ConnectionRefusedError):
            print(".", end="", flush=True)
            time.sleep(2)
    print("\n❌ فشل تشغيل الخادم في الوقت المحدد.")
    return False

# --- الخطوة 0: التأكد من وجود المسار الصحيح ---
if 'COMFYUI_DIR' not in locals() or not os.path.exists(COMFYUI_DIR):
    print("❌ خطأ: لم يتم العثور على مجلد ComfyUI.")
    print("   الرجاء التأكد من تشغيل الخلية 1 و 2 بنجاح أولاً.")
else:
    # --- الخطوة 1: تنظيف أي عمليات قديمة عالقة على البورت ---
    print("🧹 خطوة التنظيف: التأكد من أن البورت فارغ...")
    get_ipython().system(f'fuser -k {PORT}/tcp')
    time.sleep(2) # إعطاء فرصة للنظام لإغلاق العملية
    print("✅ البورت الآن جاهز للاستخدام.")

    # --- الخطوة 2: تشغيل ComfyUI في الخلفية ---
    print(f"🚀 جاري تشغيل ComfyUI في الخلفية من: {COMFYUI_DIR}")
    %cd $COMFYUI_DIR

    # تشغيل الخادم في الخلفية وتوجيه المخرجات إلى /dev/null لتجنب تجميد الخلية
    get_ipython().system_raw(f'python main.py --listen 0.0.0.0 --port {PORT} --preview-method auto > /dev/null 2>&1 &')

    # --- الخطوة 3: الانتظار الذكي للسيرفر ---
    if wait_for_service(PORT):
        # --- الخطوة 4: إنشاء الرابط العام ---
        print("\n🔗 جاري إنشاء رابط عام للوصول إلى ComfyUI...")
        try:
            get_ipython().system('pip install pyngrok --quiet')
            from pyngrok import ngrok
            # قطع أي أنفاق قديمة لضمان عدم حدوث مشاكل
            for tunnel in ngrok.get_tunnels():
                ngrok.disconnect(tunnel.public_url)

            public_url = ngrok.connect(PORT)
            print("\n" + "="*70)
            print("🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك. اضغط عليه لفتحه. 🎉🎉🎉")
            print(public_url)
            print("="*70)
        except Exception as e:
            print(f"\n❌ فشل إنشاء الرابط عبر ngrok. الخطأ: {e}")
    else:
        print("\n‼️ تعذر تشغيل ComfyUI. يرجى مراجعة الخطوات السابقة أو محاولة إعادة تشغيل البيئة.")

🧹 خطوة التنظيف: التأكد من أن البورت فارغ...
8188/tcp:             7872
✅ البورت الآن جاهز للاستخدام.
🚀 جاري تشغيل ComfyUI في الخلفية من: /content/drive/MyDrive/ComfyUI_Course_Project/ComfyUI
/content/drive/MyDrive/ComfyUI_Course_Project/ComfyUI
⏳ جاري انتظار الخادم ليكون جاهزًا على البورت 8188........
✅ الخادم يعمل الآن وجاهز للاستقبال.

🔗 جاري إنشاء رابط عام للوصول إلى ComfyUI...


ERROR:pyngrok.process.ngrok:t=2025-08-10T10:16:23+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-10T10:16:23+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-10T10:16:23+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut


❌ فشل إنشاء الرابط عبر ngrok. الخطأ: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
